In [4]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,18 hours 15 mins
H2O cluster timezone:,UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_siemanalyst_woo8vh
H2O cluster total nodes:,1
H2O cluster free memory:,4.564 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [6]:
# Import a sample binary outcome train/test set into H2O
#train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
#test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")
train = h2o.import_file("/home/siemanalyst/notebooks/siem/higgs_train_10k.csv")
test = h2o.import_file("/home/siemanalyst/notebooks/siem/higgs_test_5k.csv")

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20181113_103202,0.78935,0.552111,0.312962,0.432441,0.187005
StackedEnsemble_BestOfFamily_AutoML_20181113_103202,0.787983,0.553555,0.33571,0.433071,0.187551
XGBoost_grid_1_AutoML_20181113_103202_model_3,0.784674,0.559172,0.318019,0.435266,0.189457
XGBoost_grid_1_AutoML_20181113_103202_model_4,0.78436,0.557735,0.333439,0.435017,0.18924
XGBoost_1_AutoML_20181113_103202,0.782482,0.559651,0.329752,0.435876,0.189988
GBM_5_AutoML_20181113_103202,0.782347,0.559473,0.333556,0.435807,0.189928
XGBoost_3_AutoML_20181113_103202,0.7819,0.559799,0.321968,0.435985,0.190083
XGBoost_2_AutoML_20181113_103202,0.781374,0.559294,0.340889,0.435952,0.190054
GBM_1_AutoML_20181113_103202,0.779842,0.561592,0.309057,0.436758,0.190758
GBM_2_AutoML_20181113_103202,0.778635,0.561412,0.324663,0.437127,0.19108


In [5]:
# The leader model is stored here
aml.leader

# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

# or:
preds = aml.leader.predict(test)

preds.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1
0,0.765351,0.234649
0,0.783022,0.216978
1,0.585335,0.414665
1,0.326434,0.673566
0,0.721252,0.278748
1,0.28633,0.71367
1,0.311996,0.688004
1,0.660576,0.339424
1,0.559924,0.440076
0,0.785194,0.214806
